# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
weather_data_load = "weather_df.csv"
weather_data_1 = pd.read_csv(weather_data_load)
weather_data = weather_data_1.dropna()
del weather_data["Unnamed: 0"]
weather_data

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,takoradi,GH,2020-11-28 12:12:28,4.88,-1.76,23.0,83.0,79.592,2.83
1,ghaziabad,IN,2020-11-28 12:12:28,28.67,77.43,15.0,71.0,55.670,1.28
3,albany,US,2020-11-28 12:12:28,42.60,-73.97,65.0,59.0,48.272,0.89
4,dharmanagar,IN,2020-11-28 12:10:23,24.37,92.17,0.0,91.0,60.692,0.73
6,umm lajj,SA,2020-11-28 12:12:29,25.02,37.27,0.0,48.0,73.382,4.43
...,...,...,...,...,...,...,...,...,...
600,maniitsoq,GL,2020-11-28 12:14:34,65.42,-52.90,100.0,85.0,25.808,5.00
601,tabou,CI,2020-11-28 12:14:35,4.42,-7.35,2.0,81.0,79.844,2.42
602,bitung,ID,2020-11-28 12:14:35,1.45,125.18,40.0,100.0,75.470,1.00
603,talcahuano,CL,2020-11-28 12:13:19,-36.72,-73.12,40.0,52.0,66.470,5.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
#Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data[["Latitude", "Longitude"]]

humidity = weather_data["Humidity (%)"].astype(float)

In [32]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)
plt.savefig("../images/Heatmap.png")
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Filter vacation with zero cloudiness
vacation_no_cloud = weather_data[weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,umm lajj,SA,2020-11-28 12:12:29,25.02,37.27,0.0,48.0,73.382,4.43
1,moron,AR,2020-11-28 12:10:58,-34.65,-58.62,0.0,57.0,75.470,4.10
2,mount isa,AU,2020-11-28 12:13:05,-20.73,139.50,0.0,47.0,75.470,1.50
3,pasni,PK,2020-11-28 12:13:12,25.26,63.47,0.0,48.0,73.292,3.70
4,dwarka,IN,2020-11-28 12:13:24,22.24,68.97,0.0,46.0,75.542,6.84
5,najran,SA,2020-11-28 12:13:53,17.49,44.13,0.0,37.0,71.870,0.28
6,arlit,NE,2020-11-28 12:13:55,18.74,7.39,0.0,21.0,79.790,2.31
7,ati,TD,2020-11-28 12:14:33,13.22,18.34,0.0,17.0,79.286,4.18


In [34]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)
plt.savefig("../images/NearestHotel.png")
# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,umm lajj,SA,2020-11-28 12:12:29,25.02,37.27,0.0,48.0,73.382,4.43,Moon light Furnished Units
1,moron,AR,2020-11-28 12:10:58,-34.65,-58.62,0.0,57.0,75.470,4.10,Hotel Morón
2,mount isa,AU,2020-11-28 12:13:05,-20.73,139.50,0.0,47.0,75.470,1.50,ibis Styles Mt Isa Verona
3,pasni,PK,2020-11-28 12:13:12,25.26,63.47,0.0,48.0,73.292,3.70,3G hotel pasni
4,dwarka,IN,2020-11-28 12:13:24,22.24,68.97,0.0,46.0,75.542,6.84,Hotel Narayan Avezika Comfort
5,najran,SA,2020-11-28 12:13:53,17.49,44.13,0.0,37.0,71.870,0.28,محمد هادي ال فطيح
6,arlit,NE,2020-11-28 12:13:55,18.74,7.39,0.0,21.0,79.790,2.31,Hôtel Telwa Bungalow
7,ati,TD,2020-11-28 12:14:33,13.22,18.34,0.0,17.0,79.286,4.18,Nearest hotel not found


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

In [37]:
# Add marker layer ontop of heat map

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Save
plt.savefig("../images/PerfectVacationHeatMap.png")

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>